In [1]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from glob import glob
from PIL import Image
import numpy as np
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
from PIL import Image
import os

In [2]:
# 读取数据
train_csv_path = r"C:\Users\Vivo\2025_medicalimage_and_AI\MURA-v1.1\train_labeled_studies.csv"
valid_csv_path = r"C:\Users\Vivo\2025_medicalimage_and_AI\MURA-v1.1\valid_labeled_studies.csv"
data_root = r"C:\Users\Vivo\2025_medicalimage_and_AI"


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1)
])

# 获取病人统计信息的函数
def get_patient_stats(csv_path, mode):
    df = pd.read_csv(csv_path, header=None, names=['path', 'label'])
    stats = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        study_rel_path, label = row['path'], row['label']
        study_path = os.path.join(data_root, study_rel_path)
        image_paths = glob(os.path.join(study_path, "*.png"))

        pixel_means = []
        pixel_stds = []

        for img_path in image_paths:
            img = Image.open(img_path).convert("L")
            img = transform(img)  # 你可以用适当的 transform
            img_np = np.array(img).astype(np.float32) / 255.0
            pixel_means.append(img_np.mean())
            pixel_stds.append(img_np.std())

        patient_mean = np.mean(pixel_means)
        patient_std = np.mean(pixel_stds)
        body_part = study_rel_path.split("/")[2]

        stats.append({
            "path": study_rel_path,  # 加入 path
            "body_part": body_part,
            "label": label,
            "mean": patient_mean,
            "std": patient_std,
            "mode": mode
        })

    return pd.DataFrame(stats)


In [4]:

# 获取训练集和验证集的病人统计信息
train_stats = get_patient_stats(train_csv_path, "train")
valid_stats = get_patient_stats(valid_csv_path, "valid")

# 合并训练集和验证集的统计信息
#df = pd.concat([train_stats, valid_stats], ignore_index=True)
#print(df.head())
#print(df.shape)  # 检查数据集大小

  0%|          | 0/13457 [00:00<?, ?it/s]

100%|██████████| 1199/1199 [00:11<00:00, 104.87it/s]


In [17]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# 步骤 1：分层抽样与 KMeans + PCA 聚类
stratified = df.groupby('body_part')

df_train_new = []
df_valid_new = []

# 先依 body_part 分層
for body_part, body_group in df.groupby('body_part'):
    label_counts = body_group['label'].value_counts(normalize=True)
    label_ratio_0 = label_counts.get(0, 0)
    label_ratio_1 = label_counts.get(1, 0)

    # 根据原始比例计算验证集的目标大小
    total_samples = len(body_group)
    valid_size = int(total_samples * 0.2)  # 验证集大小为 20%

    # 计算验证集标签 0 和标签 1 的目标样本数
    valid_size_0 = int(valid_size * (label_ratio_0))
    valid_size_1 = valid_size - valid_size_0  # 剩余的验证集大小作为标签 1 的样本数

    # 计算每个标签的新倍率
    new_ratio_0 = (valid_size / 2) / valid_size_0  # 调整后倍率，确保标签 0 和标签 1 数量平衡
    new_ratio_1 = (valid_size / 2) / valid_size_1  # 调整后倍率，确保标签 0 和标签 1 数量平衡

    # 引入随机性，范围 ±0.03
    random_factor = np.random.uniform(-0.03, 0.03)

    # 调整倍率，确保两者总和不变
    adjusted_ratio_0 = new_ratio_0 + random_factor
    adjusted_ratio_1 = new_ratio_1 - random_factor

    # 计算实际抽样数量
    label_group_0 = body_group[body_group['label'] == 0]
    label_group_1 = body_group[body_group['label'] == 1]

    # 根据调整后的倍率计算每个标签的抽样数量
    label_group_valid_0 = label_group_0.sample(n=int(valid_size_0 * adjusted_ratio_0), random_state=42)
    label_group_valid_1 = label_group_1.sample(n=int(valid_size_1 * adjusted_ratio_1), random_state=42)

    # 剩余样本作为训练集
    label_group_train_0 = label_group_0.drop(label_group_valid_0.index)
    label_group_train_1 = label_group_1.drop(label_group_valid_1.index)

    # 合并训练集和验证集
    df_train_new.append(label_group_train_0)
    df_train_new.append(label_group_train_1)
    df_valid_new.append(label_group_valid_0)
    df_valid_new.append(label_group_valid_1)

# 合并最终的训练集和验证集
df_train_final = pd.concat(df_train_new, ignore_index=True)
df_valid_final = pd.concat(df_valid_new, ignore_index=True)

# 确保每行只有路径和标签
df_train_final_c = df_train_final[['path', 'label']]
df_valid_final_c = df_valid_final[['path', 'label']]

df_valid_final_c.to_csv('valid_cluster_split.csv', index=False, header=False)

'''加上household'''
# 加入 body_part 資訊（因為之前輸出的是 path 和 label，所以先從原始 df 中補上）
df_train_full = pd.merge(df_train_final_c, df[['path', 'body_part']], on='path', how='left')

# 初始化新的 train 和 household list
df_train_reduced = []
df_household_valid = []

# 依 body_part 分層抽出 1/8 作為 household valid
for body_part, group in df_train_full.groupby('body_part'):
    household_sample = group.sample(frac=1/8, random_state=42)
    remaining = group.drop(household_sample.index)

    df_household_valid.append(household_sample)
    df_train_reduced.append(remaining)

# 合併資料
df_train_final_reduce = pd.concat(df_train_reduced, ignore_index=True)
df_train_final_c_reduce = df_train_final_reduce[['path', 'label']]
df_household_final = pd.concat(df_household_valid, ignore_index=True)
df_household_final_c = df_household_final[['path', 'label']]

# 輸出為 CSV
df_train_final_c.to_csv('train_cluster_split.csv', index=False, header=False)
df_household_final_c.to_csv('household_valid.csv', index=False, header=False)

# 顯示分佈
print("新的訓練集分布：")
print(df_train_final_c['label'].value_counts())

print("\n验证集分布：")
print(df_valid_final_c['label'].value_counts())

print("\nHousehold 驗證集分布：")
print(df_household_final_c['label'].value_counts())


新的訓練集分布：
label
0    7479
1    4252
Name: count, dtype: int64

验证集分布：
label
1    1463
0    1462
Name: count, dtype: int64

Household 驗證集分布：
label
0    943
1    524
Name: count, dtype: int64


In [18]:

# 打印分布报告
def show_distribution(df_new, name):
    dist = df_new.groupby(["body_part", "label"]).size().unstack(fill_value=0)
    print(f"\n📊 {name} Data Distribution:")
    print(dist)
    print("-" * 40)

show_distribution(df_train_final_reduce, "Train")
show_distribution(df_valid_final, "Valid")
show_distribution(df_household_final, "Household Valid")
# 输出最终的 train 和 valid 数据集

print("shape of train:", df_train_final_reduce.shape)
print("shape of valid:", df_valid_final.shape)
print("shape of household valid:", df_household_final.shape)
print("✅ Done! New train/valid csv saved.")


📊 Train Data Distribution:
label           0     1
body_part              
XR_ELBOW      881   459
XR_FINGER    1009   469
XR_FOREARM    491   218
XR_HAND      1196   331
XR_HUMERUS    275   235
XR_SHOULDER  1020  1091
XR_WRIST     1664   925
----------------------------------------

📊 Valid Data Distribution:
label          0    1
body_part            
XR_ELBOW     187  193
XR_FINGER    211  210
XR_FOREARM    98  102
XR_HAND      225  215
XR_HUMERUS    71   73
XR_SHOULDER  301  301
XR_WRIST     369  369
----------------------------------------

📊 Household Valid Data Distribution:
label          0    1
body_part            
XR_ELBOW     118   74
XR_FINGER    152   59
XR_FOREARM    70   31
XR_HAND      177   41
XR_HUMERUS    43   30
XR_SHOULDER  142  160
XR_WRIST     241  129
----------------------------------------
shape of train: (10264, 3)
shape of valid: (2925, 6)
shape of household valid: (1467, 3)
✅ Done! New train/valid csv saved.


## 不使用kmeans_pca

In [15]:
import numpy as np
import pandas as pd


df_train_new = []
df_valid_new = []

# 分 stratified valid：每個 body_part 各自處理
for body_part, body_group in df.groupby('body_part'):
    # 各類別分開
    label_0 = body_group[body_group['label'] == 0]
    label_1 = body_group[body_group['label'] == 1]

    # 計算每類別該抽多少樣本（正負類各 10%，總共 valid 佔 20%）
    # 隨機性範圍 ±0.03
    random_factor = np.random.uniform(-0.03, 0.03)
    valid_size_class_0 = int(len(body_group) *(0.1 + random_factor))
    valid_size_class_1 = int(len(body_group) *(0.1 - random_factor))

    # 若數量不足，使用 min 防止錯誤
    sample_0 = label_0.sample(n=min(valid_size_class_0, len(label_0)), random_state=42)
    sample_1 = label_1.sample(n=min(valid_size_class_1, len(label_1)), random_state=42)

    # 合併成 valid
    valid_part = pd.concat([sample_0, sample_1])
    train_part = body_group.drop(valid_part.index)

    df_valid_new.append(valid_part)
    df_train_new.append(train_part)

# 合併所有 body_part 的 train 和 valid
df_train_final = pd.concat(df_train_new, ignore_index=True)
df_valid_final = pd.concat(df_valid_new, ignore_index=True)

# 保留 path 和 label 欄
df_train_final_c = df_train_final[['path', 'label']]
df_valid_final_c = df_valid_final[['path', 'label']]
df_valid_final_c.to_csv('valid_balanced_random.csv', index=False, header=False)

# household：從 train_final 中取每個 body_part 的 10%（無需平衡）
df_train_full = pd.merge(df_train_final_c, df[['path', 'body_part']], on='path', how='left')

df_train_reduced = []
df_household_valid = []

for body_part, group in df_train_full.groupby('body_part'):
    n_household = int(len(group) / 8)  # 明確表示是取1/8
    household_sample = group.sample(n=n_household, random_state=42)
    remaining = group.drop(household_sample.index)

    df_household_valid.append(household_sample)
    df_train_reduced.append(remaining)

# 合併
df_train_final_reduce = pd.concat(df_train_reduced, ignore_index=True)
df_train_final_c_reduce = df_train_final_reduce[['path', 'label']]
df_household_final = pd.concat(df_household_valid, ignore_index=True)
df_household_final_c = df_household_final[['path', 'label']]

# 輸出 CSV
df_train_final_c_reduce.to_csv('train_random_split.csv', index=False, header=False)
df_household_final_c.to_csv('household_random.csv', index=False, header=False)

# 顯示分布
print("新的訓練集分布：")
print(df_train_final_c_reduce['label'].value_counts())

print("\n验证集分布（已平衡）：")
print(df_valid_final_c['label'].value_counts())

print("\nHousehold 驗證集分布（無需平衡）：")
print(df_household_final_c['label'].value_counts())


新的訓練集分布：
label
0    6535
1    3734
Name: count, dtype: int64

验证集分布（已平衡）：
label
0    1483
1    1441
Name: count, dtype: int64

Household 驗證集分布（無需平衡）：
label
0    923
1    540
Name: count, dtype: int64


In [16]:

# 打印分布报告
def show_distribution(df_new, name):
    dist = df_new.groupby(["body_part", "label"]).size().unstack(fill_value=0)
    print(f"\n📊 {name} Data Distribution:")
    print(dist)
    print("-" * 40)

show_distribution(df_train_final_reduce, "Train")
show_distribution(df_valid_final, "Valid")
show_distribution(df_household_final, "Household Valid")
# 输出最终的 train 和 valid 数据集

print("shape of train:", df_train_final_reduce.shape)
print("shape of valid:", df_valid_final.shape)
print("shape of household valid:", df_household_final.shape)
print("✅ Done! New train/valid csv saved.")


📊 Train Data Distribution:
label           0     1
body_part              
XR_ELBOW      859   481
XR_FINGER    1026   452
XR_FOREARM    499   209
XR_HAND      1185   346
XR_HUMERUS    280   231
XR_SHOULDER  1078  1034
XR_WRIST     1608   981
----------------------------------------

📊 Valid Data Distribution:
label          0    1
body_part            
XR_ELBOW     200  181
XR_FINGER    203  218
XR_FOREARM    82  119
XR_HAND      243  193
XR_HUMERUS    73   71
XR_SHOULDER  230  372
XR_WRIST     452  287
----------------------------------------

📊 Household Valid Data Distribution:
label          0    1
body_part            
XR_ELBOW     127   64
XR_FINGER    143   68
XR_FOREARM    78   23
XR_HAND      170   48
XR_HUMERUS    36   36
XR_SHOULDER  155  146
XR_WRIST     214  155
----------------------------------------
shape of train: (10269, 3)
shape of valid: (2924, 6)
shape of household valid: (1463, 3)
✅ Done! New train/valid csv saved.


In [6]:
df_train_reduced = []
df_household_valid = []
print("train_stats shape:", train_stats['label'].value_counts())
for body_part, group in train_stats.groupby('body_part'):
    household_sample = group.sample(frac=0.1, random_state=42)

    remaining = group.drop(household_sample.index)

    df_household_valid.append(household_sample)
    df_train_reduced.append(remaining)

# 合併
df_train_final_reduce = pd.concat(df_train_reduced, ignore_index=True)
df_train_final_c_reduce = df_train_final_reduce[['path', 'label']]
df_household_final = pd.concat(df_household_valid, ignore_index=True)
df_household_final_c = df_household_final[['path', 'label']]

# 輸出 CSV
df_train_final_c_reduce.to_csv('origin_train.csv', index=False, header=False)
df_household_final_c.to_csv('household_for_real.csv', index=False, header=False)

# 顯示分布
print("新的訓練集分布：")
print(df_train_final_c_reduce['label'].value_counts())

print("\nHousehold 驗證集分布（無需平衡）：")
print(df_household_final_c['label'].value_counts())

train_stats shape: label
0    8280
1    5177
Name: count, dtype: int64
新的訓練集分布：
label
0    7463
1    4648
Name: count, dtype: int64

Household 驗證集分布（無需平衡）：
label
0    817
1    529
Name: count, dtype: int64
